In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import pandas as pd
df_final=pd.read_csv("./drive/final_combined1.csv")

In [0]:
y=df_final['label_id']

In [0]:
df_final.drop("label",axis=1,inplace=True)

In [0]:
df_final.drop("label_id",axis=1,inplace=True)

In [0]:
df_final.head()

,Unnamed: 0,Unnamed: 0.1,true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,speaker_id,job_id,party_id,state_id,subject_id,venue_id,Unnamed: 0.2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,30,30.1,42,23,18,6,2,0,41,2,1,0,-0.008703,0.000002,0.000019,-0.035479,-0.000096,-0.008818,0.000056,-0.019370,0.000015,-0.055830,-0.001470,0.000124,-0.028566,-0.000880,-0.005341,0.001807,-0.048275,0.009256,-0.011681,0.009886,-0.014376,0.001010,-0.000083,-0.016246,1.339886e-07,0.000048,...,-0.000703,-1.099579e-08,0.000804,-0.000107,0.016372,-0.063083,-0.000020,0.000483,0.001843,-0.004911,-0.047727,-0.001382,0.000345,0.026288,-0.015372,0.040351,0.000329,0.000007,-0.007211,-0.007370,0.013208,-0.000459,-0.050433,0.019824,-0.009512,0.014058,0.000017,0.001615,0.009291,0.271614,0.000002,-0.000088,0.000016,-0.054505,0.023598,0.011743,-0.005263,-6.092151e-08,-0.000072,0.023266
1,1,1,2,1.0,0,0,0,20,8,1,47,8,5,1,-0.008771,0.000002,0.000017,-0.036314,-0.000096,-0.009038,0.000057,-0.017748,0.000015,-0.057006,-0.001406,0.000126,-0.028000,-0.000862,-0.005494,0.001760,-0.048898,0.009222,-0.011670,0.009519,-0.014749,0.001063,-0.000084,-0.016013,1.380130e-07,0.000048,...,0.000608,-1.131831e-08,0.000784,-0.000102,0.018040,-0.064779,-0.000020,0.000487,0.001889,-0.004914,-0.049202,-0.001276,0.000360,0.027161,-0.015585,0.040775,0.000395,0.000007,-0.006679,-0.007087,0.013209,-0.000506,-0.052270,0.019514,-0.009536,0.015011,0.000016,0.001461,0.008935,0.262467,0.000002,-0.000087,0.000016,-0.056269,0.022417,0.012213,-0.005315,-6.177299e-08,-0.000073,0.022930
2,2,2,63,114.0,51,37,61,1,0,0,30,13,11,2,-0.008635,0.000001,0.000017,-0.036291,-0.000101,-0.008910,0.000057,-0.017106,0.000014,-0.051966,-0.000962,0.000124,-0.027530,-0.000869,-0.005653,0.001398,-0.048567,0.008645,-0.011255,0.010498,-0.014472,0.001059,-0.000098,-0.023778,1.283375e-07,0.000051,...,-0.001534,-1.474218e-08,0.000790,-0.000103,0.017543,-0.061618,-0.000020,0.000486,0.001852,-0.004884,-0.047323,-0.002176,0.000381,0.025072,-0.014236,0.043786,-0.000316,0.000007,-0.007530,-0.007681,0.013281,-0.000430,-0.050835,0.018814,-0.009781,0.014052,0.000016,0.001578,0.008907,0.265967,0.000002,-0.000090,0.000016,-0.055612,0.022782,0.010968,-0.005248,-6.212105e-08,-0.000077,0.023121
3,3,3,1,1.0,3,1,1,20,10,0,35,13,3,3,-0.008698,0.000001,0.000017,-0.035242,-0.000099,-0.008914,0.000059,-0.017113,0.000014,-0.060316,-0.000605,0.000127,-0.026840,-0.000861,-0.005567,0.001739,-0.047204,0.008665,-0.011157,0.009607,-0.014229,0.001022,-0.000108,-0.021011,1.330294e-07,0.000048,...,-0.000833,-1.122022e-08,0.000822,-0.000100,0.017131,-0.064085,-0.000020,0.000490,0.001879,-0.004871,-0.048211,-0.001819,0.000350,0.026944,-0.015236,0.041141,0.000944,0.000007,-0.007107,-0.006950,0.013261,-0.000462,-0.051212,0.019365,-0.009558,0.013587,0.000017,0.001601,0.008807,0.272021,0.000002,-0.000085,0.000016,-0.054348,0.022007,0.011040,-0.005224,-6.216437e-08,-0.000069,0.022895
4,4,4,5,7.0,2,2,7,20,10,1,47,13,11,4,-0.008659,0.000002,0.000016,-0.035649,-0.000103,-0.008788,0.000056,-0.017339,0.000015,-0.057339,-0.001244,0.000127,-0.026801,-0.000861,-0.005994,0.003094,-0.045458,0.007962,-0.011279,0.009924,-0.014186,0.001031,-0.000085,-0.009360,3.856394e-07,0.000071,...,-0.000191,-1.171954e-08,0.000739,-0.000103,0.017796,-0.065653,-0.000022,0.000498,0.001925,-0.004923,-0.049953,-0.001748,0.000436,0.027311,-0.016354,0.042573,0.000311,0.000007,-0.007206,-0.007535,0.013188,-0.000506,-0.049636,0.018632,-0.009433,0.015441,0.000017,0.001620,0.008784,0.270773,0.000003,-0.000085,0.000016,-0.057452,0.022379,0.013802,-0.005138,-6.003636e-08,-0.000068,0.023294


In [0]:
df_final = df_final.loc[:, ~df_final.columns.str.contains('^Unnamed')]

In [0]:
df_final.head()

,true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,speaker_id,job_id,party_id,state_id,subject_id,venue_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,30,30.1,42,23,18,6,2,0,41,2,1,-0.008703,0.000002,0.000019,-0.035479,-0.000096,-0.008818,0.000056,-0.019370,0.000015,-0.055830,-0.001470,0.000124,-0.028566,-0.000880,-0.005341,0.001807,-0.048275,0.009256,-0.011681,0.009886,-0.014376,0.001010,-0.000083,-0.016246,1.339886e-07,0.000048,0.029436,0.000040,-0.000011,...,-0.000703,-1.099579e-08,0.000804,-0.000107,0.016372,-0.063083,-0.000020,0.000483,0.001843,-0.004911,-0.047727,-0.001382,0.000345,0.026288,-0.015372,0.040351,0.000329,0.000007,-0.007211,-0.007370,0.013208,-0.000459,-0.050433,0.019824,-0.009512,0.014058,0.000017,0.001615,0.009291,0.271614,0.000002,-0.000088,0.000016,-0.054505,0.023598,0.011743,-0.005263,-6.092151e-08,-0.000072,0.023266
1,2,1.0,0,0,0,20,8,1,47,8,5,-0.008771,0.000002,0.000017,-0.036314,-0.000096,-0.009038,0.000057,-0.017748,0.000015,-0.057006,-0.001406,0.000126,-0.028000,-0.000862,-0.005494,0.001760,-0.048898,0.009222,-0.011670,0.009519,-0.014749,0.001063,-0.000084,-0.016013,1.380130e-07,0.000048,0.029697,0.000040,-0.000010,...,0.000608,-1.131831e-08,0.000784,-0.000102,0.018040,-0.064779,-0.000020,0.000487,0.001889,-0.004914,-0.049202,-0.001276,0.000360,0.027161,-0.015585,0.040775,0.000395,0.000007,-0.006679,-0.007087,0.013209,-0.000506,-0.052270,0.019514,-0.009536,0.015011,0.000016,0.001461,0.008935,0.262467,0.000002,-0.000087,0.000016,-0.056269,0.022417,0.012213,-0.005315,-6.177299e-08,-0.000073,0.022930
2,63,114.0,51,37,61,1,0,0,30,13,11,-0.008635,0.000001,0.000017,-0.036291,-0.000101,-0.008910,0.000057,-0.017106,0.000014,-0.051966,-0.000962,0.000124,-0.027530,-0.000869,-0.005653,0.001398,-0.048567,0.008645,-0.011255,0.010498,-0.014472,0.001059,-0.000098,-0.023778,1.283375e-07,0.000051,0.029427,0.000040,-0.000011,...,-0.001534,-1.474218e-08,0.000790,-0.000103,0.017543,-0.061618,-0.000020,0.000486,0.001852,-0.004884,-0.047323,-0.002176,0.000381,0.025072,-0.014236,0.043786,-0.000316,0.000007,-0.007530,-0.007681,0.013281,-0.000430,-0.050835,0.018814,-0.009781,0.014052,0.000016,0.001578,0.008907,0.265967,0.000002,-0.000090,0.000016,-0.055612,0.022782,0.010968,-0.005248,-6.212105e-08,-0.000077,0.023121
3,1,1.0,3,1,1,20,10,0,35,13,3,-0.008698,0.000001,0.000017,-0.035242,-0.000099,-0.008914,0.000059,-0.017113,0.000014,-0.060316,-0.000605,0.000127,-0.026840,-0.000861,-0.005567,0.001739,-0.047204,0.008665,-0.011157,0.009607,-0.014229,0.001022,-0.000108,-0.021011,1.330294e-07,0.000048,0.029639,0.000040,-0.000012,...,-0.000833,-1.122022e-08,0.000822,-0.000100,0.017131,-0.064085,-0.000020,0.000490,0.001879,-0.004871,-0.048211,-0.001819,0.000350,0.026944,-0.015236,0.041141,0.000944,0.000007,-0.007107,-0.006950,0.013261,-0.000462,-0.051212,0.019365,-0.009558,0.013587,0.000017,0.001601,0.008807,0.272021,0.000002,-0.000085,0.000016,-0.054348,0.022007,0.011040,-0.005224,-6.216437e-08,-0.000069,0.022895
4,5,7.0,2,2,7,20,10,1,47,13,11,-0.008659,0.000002,0.000016,-0.035649,-0.000103,-0.008788,0.000056,-0.017339,0.000015,-0.057339,-0.001244,0.000127,-0.026801,-0.000861,-0.005994,0.003094,-0.045458,0.007962,-0.011279,0.009924,-0.014186,0.001031,-0.000085,-0.009360,3.856394e-07,0.000071,0.029522,0.000039,-0.000011,...,-0.000191,-1.171954e-08,0.000739,-0.000103,0.017796,-0.065653,-0.000022,0.000498,0.001925,-0.004923,-0.049953,-0.001748,0.000436,0.027311,-0.016354,0.042573,0.000311,0.000007,-0.007206,-0.007535,0.013188,-0.000506,-0.049636,0.018632,-0.009433,0.015441,0.000017,0.001620,0.008784,0.270773,0.000003,-0.000085,0.000016,-0.057452,0.022379,0.013802,-0.005138,-6.003636e-08,-0.000068,0.023294


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_final, y, test_size=0.33, random_state=16)

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def svm_wrapper(X_train,Y_train):
    param_grid = [
    {'C': [1, 10, 100], 'kernel': ['linear']},
    {'C': [1, 10,100], 'gamma': [0.1,0.01,0.001], 'kernel': ['rbf']},]
    svm = GridSearchCV(SVC(),param_grid)
    svm.fit(X_train, y_train)
    return(svm)

# svm
svm = svm_wrapper(X_train,y_train)
y_pred = svm.predict(X_test)
score = accuracy_score(y_test,y_pred)
print("accuarcy :", score)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


accuarcy : 0.3794749403341289


In [0]:
label_dict = {'pants-fire':0,'false':0,'barely-true':0,'half-true':1,'mostly-true':1,'true':1}

In [0]:
y_binary=y
  

In [0]:
for i in range(len(y_binary)):
  if y_binary[i] == 0 or y_binary[i] ==1 or y_binary[i]== 2:
    y_binary[i]=0
  else:
    y_binary[i]=1

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_final, y_binary, test_size=0.33, random_state=16)

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def svm_wrapper(X_train,Y_train):
    param_grid = [
    {'C': [1, 10, 100,1000], 'kernel': ['linear']},
    {'C': [1, 10,100,1000], 'gamma': [0.1,0.01,0.001,0.0001], 'kernel': ['rbf']},]
    svm = GridSearchCV(SVC(),param_grid)
    svm.fit(X_train, y_train)
    return(svm)

# svm
svm = svm_wrapper(X_train,y_train)
y_pred = svm.predict(X_test)
score = accuracy_score(y_test,y_pred)
print("accuarcy :", score)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


accuarcy : 0.6897374701670644


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_final, y_binary, test_size=0.33, random_state=16)

In [0]:
rfc=RandomForestClassifier(random_state=42)

param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

CV_rfc.best_params_

rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=8, criterion='gini')
rfc1.fit(X_train, y_train)
pred=rfc1.predict(X_test)
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

Accuracy for Random Forest on CV data:  0.5656324582338902
